# 1.import声明

In [1]:
from tqdm import tqdm
import random
import time
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import CrossEntropyLoss,BCEWithLogitsLoss
from torch.nn.utils.rnn import pad_sequence
from transformers  import BertModel,BertConfig,BertTokenizer

from torch import optim
from torch.autograd import Variable

from sklearn.metrics import roc_auc_score  
from sklearn.model_selection import KFold

# 2.数据采样

In [2]:
train = pd.read_csv('./data/bert/15_10_con/train_pair.csv')
dev = pd.read_csv('./data/bert/15_10_con/dev_pair.csv')
test = pd.read_csv('./data/bert/15_10_con/test_pair.csv')


In [3]:
train.iloc[1232]['0']

'[CLS] d.a. seeks murder charge against former fort worth officer who former nfl lineman justin bannan arrested for attempted murder 0 fisherman faces prison fine for cruel act on rare fish serial stowaway arrested at chicago airport for 2nd time 0 insiders predict nfl week 8 winners 0 0 0 0 gabbard hits back at queen of warmongers clinton 0 0 teen breaks into german prison in bid to win back harry dunns family launches legal action against the uk foreign fact checker trumps shiny new talking point about income growth police arrest motorcyclist who led high-speed chase and went home californias legal weed profits going up in smoke 0 0 the woman who flipped off trump has won an election woman 78 gets 22 years for attempted murder of lawyer [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [SEP] how russia meddles abroad for profit cash trolls and a [SEP]'

# 3.数据token

In [4]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
def get_dummies(l, size=2):
    res = list()
    for i in l:
        tmp = [0] * size
        tmp[i] = 1
        res.append(tmp)
    return res
def token(data):
    arr = data['0'].values.tolist()
    
    
    input_ids = [tokenizer.convert_tokens_to_ids(i.split(' ')) for i in tqdm(arr)]
    input_labels = data['1'].values.tolist()
    return [input_ids,input_labels]

train_data = token(train)
dev_data = token(dev)
test_data = token(test)



100%|██████████| 49956/49956 [00:16<00:00, 2996.96it/s]


# 4.数据加载器

In [5]:
dev_set = TensorDataset(torch.LongTensor(dev_data[0]),torch.FloatTensor(get_dummies(dev_data[1])))
dev_loader = DataLoader(dataset=dev_set,batch_size=32,shuffle=True)

test_set = TensorDataset(torch.LongTensor(test_data[0][:10000]),torch.FloatTensor(get_dummies(test_data[1][:10000])))
test_loader = DataLoader(dataset=test_set,batch_size=32 ,shuffle=True)

# 5.模型构建

In [6]:
torch.cuda.empty_cache()
model_name = 'bert-base-uncased'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Bert_model(nn.Module):
    def __init__(self):
        super(Bert_model,self).__init__()
        self.model = BertModel.from_pretrained(
            "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = False, # Whether the model returns all hidden-states.
        )
        self.model.to(device)
        self.dropout = nn.Dropout(0.1)
        self.layer1 = nn.Linear(768,2)

    def forward(self,x,attention_mask=None,labels=None):
        output = self.model(x,attention_mask=attention_mask)
        x = output[1] #???
        
        x = self.layer1(x)
        
        return x
def predict(logits):
    res = torch.argmax(logits, 1)
    return res

model = Bert_model()
model.to(device)
model=nn.DataParallel(model,device_ids=[0,1]) # multi-GPU


criterion = nn.BCELoss()
sigmoid = nn.Sigmoid()
optimizer = optim.Adam(model.parameters(), lr=1e-5)


start = time.time()
accumulation_steps =8
epoch = 3
loss_collect = []

for i in range(epoch) :
    
    lossing = 0
    aucing = 0
    torch.cuda.empty_cache()
    model.train()
    x = [x for x in range(len(train_data[0]))]
    random.seed(2020 + i)
    random.shuffle(x)
    train_set = TensorDataset(torch.LongTensor(np.array(train_data[0])[x]),torch.FloatTensor(get_dummies(np.array(train_data[1])[x])))
    train_loader = DataLoader(dataset=train_set,batch_size=32,shuffle=True)
    for batch_idx,(data,target) in enumerate(train_loader):
        data,target = Variable(data).to(device),Variable(target).to(device)
       
        mask = []
        for sample in data:
            
            mask.append([1 if i != 0 else 0 for i in sample])
        
        mask =torch.Tensor(mask).to(device)
           
        output = model(data,attention_mask = mask)
        pred = predict(output)

        loss = criterion(sigmoid(output), target)

        # 梯度积累
        loss = loss/accumulation_steps
        loss.backward()
        loss_collect.append(loss.item())
#         print("\r%f" % loss, end='')
        
        if((batch_idx+1) % accumulation_steps) == 0:
            # 每  次更新一下网络中的参数
            optimizer.step()
            optimizer.zero_grad() 
        lossing += loss.item()   
        aucing += roc_auc_score(target.data.cpu().numpy().tolist(),get_dummies(pred.data.cpu().numpy().tolist()))
        if ((batch_idx+1) % accumulation_steps) == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}\tAuc:{:.6f}'.format(
                i+1, batch_idx, len(train_loader), 100. *
                batch_idx/len(train_loader), lossing/(batch_idx+1) * accumulation_steps,
                aucing/(batch_idx+1)
            ))
    torch.save(model, './data/model/model_{}.pkl'.format(i))

    
print('训练时间：', time.time()-start)
               
               

Train Epoch: 1 [7/1250 (1%)]	Loss:0.711488	Auc:0.500000
Train Epoch: 1 [15/1250 (1%)]	Loss:0.704543	Auc:0.500000
Train Epoch: 1 [23/1250 (2%)]	Loss:0.699141	Auc:0.500000
Train Epoch: 1 [31/1250 (2%)]	Loss:0.697701	Auc:0.500000
Train Epoch: 1 [39/1250 (3%)]	Loss:0.697611	Auc:0.500000


KeyboardInterrupt: 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
model_name = 'bert-base-uncased'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Bert_model(nn.Module):
    def __init__(self):
        super(Bert_model,self).__init__()
        self.model = BertModel.from_pretrained(
            "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
            num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = False, # Whether the model returns all hidden-states.
        )
        self.model.to(device)
        self.dropout = nn.Dropout(0.1)
        self.layer1 = nn.Linear(768,2)

    def forward(self,x,attention_mask=None,labels=None):
        output = self.model(x,attention_mask=attention_mask)
        x = output[1] #???
        
        
        x = self.layer1(x)
        
        return x
def predict(logits):
    res = torch.argmax(logits, 1)
    return res
from tqdm import tqdm_notebook as tqdm


correct = 0
total = 0
real = []
pre = []
for index in range(3):
    torch.cuda.empty_cache()
    model = torch.load( './data/model/model_{}.pkl'.format(index))   
    model.eval()
    for batch_idx, (data, target) in enumerate(tqdm(test_loader)):
        data = data.to(device)
        target = target.long().to(device)

        mask = []
        for sample in data:
            mask.append([1 if i != 0 else 0 for i in sample])
        mask = torch.Tensor(mask).to(device)
    
        output = model(data, attention_mask=mask)
        pred = predict(output).data.cpu().numpy().tolist()
        real.extend(target.data.cpu().numpy().tolist())
        pre.extend(pred)
    
    
    # 准确率应该达到百分之 90 以上
    score = roc_auc_score((real),get_dummies(pre))
    print(score)

In [ ]:
# score = roc_auc_score((real),get_dummies(pre))
# print(score)

In [ ]:
768 *32

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

hidden_dim = 256
embedding_dim = 300
vocab_size = 10000

class bigru_attention(BasicModule):
    def __init__(self):
        super(bigru_attention, self).__init__()
        self.hidden_dim = hidden_dim
        self.gru_layers = 2

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        # 双向GRU，//操作为了与后面的Attention操作维度匹配，hidden_dim要取偶数！
        self.bigru = nn.GRU(embedding_dim, hidden_dim // 2, num_layers=gru_layers, bidirectional=True)
        # 由nn.Parameter定义的变量都为requires_grad=True状态
        self.weight_W = nn.Parameter(torch.Tensor(hidden_dim, hidden_dim))
        self.weight_proj = nn.Parameter(torch.Tensor(hidden_dim, 1))
        # 二分类
        self.fc = nn.Linear(hidden_dim, 2)

        nn.init.uniform_(self.weight_W, -0.1, 0.1)
        nn.init.uniform_(self.weight_proj, -0.1, 0.1)

    def forward(self, sentence):
        embeds = self.embedding(sentence) # [seq_len, bs, emb_dim]
        gru_out, _ = self.bigru(embeds) # [seq_len, bs, hid_dim]
        x = gru_out.permute(1, 0, 2)
        # # # Attention过程，与上图中三个公式对应
        u = torch.tanh(torch.matmul(x, self.weight_W))
        att = torch.matmul(u, self.weight_proj)
        att_score = F.softmax(att, dim=1)
        scored_x = x * att_score
        # # # Attention过程结束
        
        feat = torch.sum(scored_x, dim=1)
        y = self.fc(feat)
        return y